In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing

In [5]:
(train_data, train_targets), (test_data, test_targets)=boston_housing.load_data()

In [11]:
print(train_data[0])
print(train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
15.2


In [15]:
print("shape of train_data:",train_data.shape )
print("shape of test_data:",test_data.shape )


shape of train_data: (404, 13)
shape of test_data: (102, 13)


In [22]:
mean = train_data.mean(axis=0)
print("mean:", mean)
train_data -= mean
print("train_data:", train_data)
std = train_data.std(axis=0)
print("std:", std)
train_data /= std
print("train_data:", train_data)
test_data -= mean
test_data /= std

mean: [ 7.83201886e-18 -1.20915379e-17 -1.97861529e-17  2.08853836e-16
  4.39692287e-18  7.41980734e-18  1.09923072e-17 -1.09923072e-18
 -2.72059603e-17 -2.30838451e-17  2.47326911e-17 -1.23663456e-18
 -3.29769215e-18]
train_data: [[-0.27224633 -0.48361547 -0.43576161 ...  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 ... -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  ...  0.78447637  0.22061726
  -1.30850006]
 ...
 [-0.40202987  0.99079651 -0.7415148  ... -0.71712291  0.07943894
  -0.67776904]
 [-0.17292018 -0.48361547  1.24588095 ... -1.71818909 -0.98764362
   0.42083466]
 [-0.40422614  2.04394792 -1.20161456 ... -1.30866202  0.23317118
  -1.15392266]]
std: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
train_data: [[-0.27224633 -0.48361547 -0.43576161 ...  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 ... -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  ...  0.78447637  0.